# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

# Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

Прочитаем файл data.csv и сохраним его в переменной df

In [2]:
df = pd.read_csv('/datasets/data.csv')

Получим первые 50 строк таблицы

In [46]:
df.head(50)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,purpose_categ,income_categ
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье \n,жилье,очень высокий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль \n,автомобиль,средний
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье \n,жилье,высокий
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование \n,образование,очень высокий
4,0,14600,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба \n,свадьба,высокий
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,покупка жилье \n,жилье,очень высокий
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,операция с жилье \n,жилье,очень высокий
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование \n,образование,средний
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,на проведение свадьба \n,свадьба,низкий
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,покупка жилье для семья \n,жилье,высокий


При анализе этих строк таблицы наблюдаем следующие проблемы:
1)отрицательные значения в столбце 'days_employed'
2)разный регистр данных в столбцах 'family_status' и 'education'
Выведем общую информацию о данных таблицы

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Проанализируем полученную информацию о таблице
1) Всего в таблице 12 столбцов с разными типами данных
2) Количество данных в разных столбцах различно - то есть в таблице есть пропущенные значения
Значения в столбцах 'days_employed'и 'total_income' следует привести к целочисленному значению - нет необходимости в такой точности знаков после запятой.
Так как пропуцщенные значения есть только в столбцах 'days_employed' и 'total_income', убедимся в том, во всех ли строках датафрейма  с пропущенными значениями отсутствует инфморация об этих значениях.

In [5]:
df[(df['days_employed'].isnull()==True) & (df['total_income'].isnull()==True)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
children            2174 non-null int64
days_employed       0 non-null float64
dob_years           2174 non-null int64
education           2174 non-null object
education_id        2174 non-null int64
family_status       2174 non-null object
family_status_id    2174 non-null int64
gender              2174 non-null object
income_type         2174 non-null object
debt                2174 non-null int64
total_income        0 non-null float64
purpose             2174 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


Предложение оказалось верным. Теперь проверим, из одной ли категории занятости эти люди, чтобы можно было заполнить пропущенные значения средним или медианным значением по группе.

In [6]:
df[(df['days_employed'].isnull()==True) & (df['total_income'].isnull()==True)]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Предположение не подтвердилось, значит при дальнейшей преобработке данных необходимо будет заменить пропущенные значения в столбцах срединим либо медианным значением по каждой из групп в отдельности.
Продолжаем смотреть информацию о таблице.

In [7]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Видны следующие странные значения
1) Минимальное значение в столбце 'children' = -1, скорее всего, это опечатка и вместо 1 было вбито -1
2) Максимальное значение в столбце 'children' = 20, что невозможно по физиологическим причинам.
3) Среднее значение в столбце 'days_employed' при переводе в года составляет 175 лет. Скорее всего, в столбце присутствуют "артефакты"
4) Минимальное значение в столбце 'dob_years' = 0. Скорее всего, возраст был пропущен или ошибочно указан нулевым.
Выведем данным по столбцам 'children','days_employed' и 'dob_years' чтобы определить, насколько часто в датафрейме встречются вышеуказанные аномалии

In [8]:
print('Количество строк с положительным трудовым стажем:',df[df['days_employed']>0].count()[1])
print('Количество строк с отрицательным трудовым стажем:',df[df['days_employed']<0].count()[1])

Количество строк с положительным трудовым стажем: 3445
Количество строк с отрицательным трудовым стажем: 15906


Отрицательный трудовой стаж - скорее всего значения, ошибочно взятые со знаком минус. Возьмём значения по модулю.

In [9]:
print('Количество людей с -1 ребёнком:',df[df['children']==-1].count()[0])
print('Количество людей с 20 детьми:',df[df['children']==20].count()[0])
print('Количество людей с нулевым возрастом:',df[df['dob_years']==0].count()[2])

Количество людей с -1 ребёнком: 47
Количество людей с 20 детьми: 76
Количество людей с нулевым возрастом: 101


Нужно будет исправить -1 ребёнка на 1
20 детей - на 2
Нулевые значения удалить или усреднить.
Получим перечень названия столбцов для проверки однородности их написания

In [10]:
df.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

Названия столбцов написаны в едином регистре, без пробелов.

# **Вывод**

Каждая строка в таблице содержит информацию о клиенте, намеревающемся взять в банке кредит. Чтобы ответить на поставленные вопросы, нужно решить несколько задач, а именно - обработка пропусков и "артефактов", лемматизация по целям получения кредита.

# Шаг 2. Предобработка данных

### Обработка пропусков

Сначала переведем все значения в столбцах 'education' и 'family_status'в нижний регистр

In [11]:
df['education']=df['education'].str.lower()
df['family_status']=df['family_status'].str.lower()

Наблюдаем артефакты - например, отрицательное число детей, нулевой возраст клиентов или слишком большой трудовой стаж(столько не живут). Обработаем их

In [12]:
df['children']=df['children'].replace(-1,1)
df['education']=df['education'].str.lower()
df['family_status']=df['family_status'].str.lower()
df['children']=df['children'].replace(20,2)

Для того, чтобы разобраться с аномально большим трудовым стажем, проведём группировку столбца по типу занятости и определим медианное значение трудового стажа для каждой из групп

In [13]:
df.groupby('income_type')['days_employed'].median()

income_type
безработный        366413.652744
в декрете           -3296.759962
госслужащий         -2689.368353
компаньон           -1547.382223
пенсионер          365213.306266
предприниматель      -520.848083
сотрудник           -1574.202821
студент              -578.751554
Name: days_employed, dtype: float64

Аномально большие положительные значения наблюдаются у безработных и пенсионеров. Видно, что никакие операции с ними не приведут к адекватному результату. Положим, что эти данные были получены по ошибке и зададим пограничное значение для этих групп - например, для пенсионеров в 40 лет (14600 дней) и менее 5 лет для безработных (1825 дней)
Возьмём значения по модулю

In [14]:
df['days_employed']=df['days_employed'].abs()

Создадим вспомогательный столбец bool, приимающий значение 1, если стаж более 14600 и 0 в остальных случаях

In [15]:
df['bool']= df.apply(lambda x:1 if x['income_type']=='пенсионер'and x['days_employed']>14600 else 0, axis = 1)

Заменим все аномальные значения трудового стажа у пенсионеров на 14600 и удалим столбец

In [16]:
df.loc[df['bool']==1,'days_employed']=14600
df = df.drop('bool',axis = 1) 

Аналогично обработаем аномалию у безработных

In [17]:
df['bool'] = df.apply(lambda x: 1 if x['income_type']=='безработный' and x['days_employed']>1825 else 0, axis = 1)
df.loc[df['bool']==1,'days_employed']= 1825
df = df.drop('bool',axis = 1) 

Проверим обработку значений

In [18]:
df.groupby('income_type')['days_employed'].median()

income_type
безработный         1825.000000
в декрете           3296.759962
госслужащий         2689.368353
компаньон           1547.382223
пенсионер          14600.000000
предприниматель      520.848083
сотрудник           1574.202821
студент              578.751554
Name: days_employed, dtype: float64

Значения обработаны, аномально больших значений больше нет. Заполним нулевые значения в столцах days_employed,total_income и dob_years медианными значениями по типу занятости клиентов.

In [19]:
df.groupby('income_type').agg({'dob_years':['median','count'],'days_employed':['median','count'],'total_income':['median','count']})

dob_years        days_employed          total_income       
                   median  count        median  count         median  count
income_type                                                                
безработный          38.0      2   1825.000000      2  131339.751676      2
в декрете            39.0      1   3296.759962      1   53829.130729      1
госслужащий          40.0   1459   2689.368353   1312  150447.935283   1312
компаньон            39.0   5085   1547.382223   4577  172357.950966   4577
пенсионер            60.0   3856  14600.000000   3443  118514.486412   3443
предприниматель      42.5      2    520.848083      1  499163.144947      1
сотрудник            39.0  11119   1574.202821  10014  142594.396847  10014
студент              22.0      1    578.751554      1   98201.625314      1

Создадим функции для каждого из столбцов, которые будут заполнять пустые значения в них медианными.

In [20]:
df['days_employed'] = df.groupby('income_type')['days_employed'].apply(lambda x: x.fillna(x.median()))
df['total_income'] = df.groupby('income_type')['total_income'].apply(lambda x: x.fillna(x.median()))
df['dob_years'] = df.groupby('income_type')['dob_years'].apply(lambda x: x.fillna(x.median()))

Проверим, остались ли нулевые строки

In [21]:
df.describe()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


# **Вывод**

Все нулевые значения заполненые медианными, исходя из типа занятости каждого клиента. Неверные значения откорректированы.

### Замена типа данных

В столбцах 'days_empoloyed'и 'total_income' произведем перевод из дробных значений в целые.

In [22]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')

Проверим 

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


Выведем первые 50 строк таблицы

In [24]:
df.head(50)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14600,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


# **Вывод**

Данные датафрейма предоставлены в читабельном виде. Столбцы 'days_employed' и 'total_income' переведены в целочисленные.

### Обработка дубликатов

Удалим все дубликаты из таблицы:

In [25]:
df= df.drop_duplicates().reset_index(drop = True)

Проверим, не осталось ли дубликатов

In [26]:
df.duplicated().sum()

0

# **Вывод**

Были удалены все дубликаты из таблицы.

### Лемматизация

Cначала выведем список уникальных значениц в столбце "purpose":

In [27]:
df['purpose'].value_counts()
m = Mystem()
from collections import Counter

Пишем функцию принимающую строку и возвращающую список лемм

In [28]:
def lemmatize_row(row): 
    lemmas = ' '.join(m.lemmatize(row))
    return(lemmas)
df['purpose_lemmas'] = df['purpose'].apply(lemmatize_row)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье \n
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль \n
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье \n
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование \n
4,0,14600,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба \n


Просматриваем уникальные значения полученных лемм

In [29]:
df['purpose_lemmas'].value_counts()

автомобиль \n                                    972
свадьба \n                                       791
на   проведение   свадьба \n                     768
сыграть   свадьба \n                             765
операция   с   недвижимость \n                   675
покупка   коммерческий   недвижимость \n         661
операция   с   жилье \n                          652
покупка   жилье   для   сдача \n                 651
операция   с   коммерческий   недвижимость \n    650
жилье \n                                         646
покупка   жилье \n                               646
покупка   жилье   для   семья \n                 638
строительство   собственный   недвижимость \n    635
недвижимость \n                                  633
операция   со   свой   недвижимость \n           627
строительство   жилой   недвижимость \n          624
покупка   недвижимость \n                        621
покупка   свой   жилье \n                        620
строительство   недвижимость \n               

# **Вывод**

Была проведена лемматизация целей получения кредита и составлен словарь из набилее часто встречающихся лемм для следующего этапа категоризации датафрейма.

Для того, чтобы провести категоризацию, разобъем данные столбцы по следующем признакам:

In [30]:
def purpose_categ(row):
    if 'автомобиль' in row:
        return('автомобиль')
    elif 'свадьба'in row:
        return('свадьба')
    elif 'образование' in row:
        return('образование')
    elif 'коммерческий' and 'недвижимость' in row:
        return('коммерческая недвижимость')
    elif 'жилье' in row or 'недвижимость' in row:
        return('жилье')  

Добавляем столбец с категориями целей получения кредита

In [31]:
df['purpose_categ'] = df['purpose_lemmas'].apply(purpose_categ)

Проверяем результат

In [32]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,purpose_categ
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье \n,жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль \n,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье \n,жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование \n,образование
4,0,14600,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба \n,свадьба


Определим квантили по которым распределяются заемщики в зависимости от уровня дохода, присвоим каждому квантилю метку и сохраним ее как значение в новом столбце, который будет характеризовать уровень дохода заемщика.

In [45]:
df['income_categ'] = pd.qcut(df['total_income'], q=4,
                                  labels = ['низкий', 'средний', 'высокий', 'очень высокий'])
new_df = df[['children', 'family_status', 'income_categ', 'debt', 'purpose_categ']]
new_df.head()

,children,family_status,income_categ,debt,purpose_categ
0,1,женат / замужем,очень высокий,0,жилье
1,1,женат / замужем,средний,0,автомобиль
2,0,женат / замужем,высокий,0,жилье
3,3,женат / замужем,очень высокий,0,образование
4,0,гражданский брак,высокий,0,свадьба


Сформируем сводную таблицу для общего представления количества просрочек по кредитам в разрезе целей кредита и в зависимости от уровня дохода и семейного положения заемщика

In [34]:
new_df_categ = new_df.pivot_table(index = 'purpose_categ',
                                             columns = ['family_status', 'income_categ'],
                                             values = 'debt',
                                             aggfunc = 'sum', fill_value = 0).stack()

Сформируем сводную таблицу для общего представления количества просрочек по кредитам в разрезе целей кредита и в зависимости от уровня дохода и количества детей

In [35]:
new_df_children = new_df.pivot_table(index = 'purpose_categ',
                                             columns = [ 'income_categ', 'children'],
                                             values = 'debt',
                                             aggfunc = 'sum').stack()

Построим сводную таблицу с указанием количества просрочек по кредиту относительно количества детей у заемщика и доли данных просрочек также относительно количества детей у заемщика.
Отсортируем сводную таблицу по размеру доли просрочек в порядке убывания.

In [36]:
new_df_children = new_df.groupby('children')['debt'].agg(['sum', 'count', 'mean']).reset_index()
new_df_children.sort_values(by = 'mean', ascending = False).rename(columns = {'mean': 'share_debt'})

,children,sum,count,share_debt
4,4,4,41,0.097561
2,2,202,2128,0.094925
1,1,445,4855,0.091658
3,3,27,330,0.081818
0,0,1063,14091,0.075438
5,5,0,9,0.000000


Построим сводную таблицу с указанием количества и доли просрочек по кредиту в зависимости от семейного статуса заемщика.
Отсортируем сводную таблицу по размеру доли просрочек в порядке убывания.

In [37]:
new_df_family_status = new_df.groupby('family_status')['debt'].agg(['sum', 'count', 'mean']).reset_index() 
new_df_family_status.sort_values(by = 'mean', ascending = False).rename(columns = {'mean': 'share_debt'})

,family_status,sum,count,share_debt
4,не женат / не замужем,274,2810,0.097509
2,гражданский брак,388,4151,0.093471
3,женат / замужем,931,12339,0.075452
0,в разводе,85,1195,0.071130
1,вдовец / вдова,63,959,0.065693


Построим сводную таблицу с указанием количества и доли просрочек по кредиту в зависимости от уровня дохода заемщика. Отсортируем сводную таблицу по размеру доли просрочек в порядке убывания.

In [38]:
df_income_status = new_df.groupby('income_categ')['debt'].agg(['sum', 'count', 'mean']).reset_index()
df_income_status.sort_values(by = 'mean', ascending = False).rename(columns = {'mean': 'share_debt'})

,income_categ,sum,count,share_debt
1,средний,483,5479,0.088155
2,высокий,448,5247,0.085382
0,низкий,427,5364,0.079605
3,очень высокий,383,5364,0.071402


Построим сводную таблицу с указанием количества и доли просрочек по кредиту в зависимости от цели кредита.
Отсортируем сводную таблицу по размеру доли просрочек в порядке убывания.

In [39]:
new_df_purpose_category = new_df.groupby('purpose_categ')['debt'].agg(['sum', 'count', 'mean']).reset_index()
new_df_purpose_category.sort_values(by = 'mean', ascending = False).rename(columns = {'mean': 'share_debt'})

,purpose_categ,sum,count,share_debt
0,автомобиль,403,4306,0.093590
3,образование,370,4013,0.092200
4,свадьба,186,2324,0.080034
2,коммерческая недвижимость,474,6351,0.074634
1,жилье,308,4460,0.069058


# **Вывод**

Можно сделать вывод о наличии зависимости между целью кредита и количеством просрочек.
Доля просрочек по кредиту в случае его оформления для приобретения жилья и недвижимости меньше по сравнению с кредитом на приобретение автомобиля и оплаты образования примерно на 2%.

# Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Сгруппируем значения задолженности по количеству детей у клиента и возьмём среднее значение в каждой из категорий:

In [40]:
df[['children','debt']].groupby('children').mean()

,debt
children,
0,0.075438
1,0.091658
2,0.094925
3,0.081818
4,0.097561
5,0.000000


# **Вывод**

Бездетные клиенты более склонны к возврату кредита в срок.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Сгруппируем значения задолженности по семейному положению клиентра и возьмём среднее значение в каждой из категорий

In [41]:
df[['family_status','debt']].groupby('family_status').mean()

,debt
family_status,
в разводе,0.071130
вдовец / вдова,0.065693
гражданский брак,0.093471
женат / замужем,0.075452
не женат / не замужем,0.097509


# **Вывод**

Наиболее склонны к просрочке кредита люди любо не женатые, либо состоящие в гражданском браке. Женатые, вдовцы и состоящие в разводе чаще возвращают кредит в срок.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Сгруппируем значения задолженности по уровню доходов клиента и возьмём среднее значение в каждой из категорий

In [42]:
df[['total_income','debt']].groupby('total_income').mean()

,debt
total_income,
20667,1.0
21205,0.0
21367,0.0
21695,0.0
21895,0.0
...,...
1711309,0.0
1715018,0.0
1726276,0.0


# **Вывод**

Самые бедные и самые богатые клиенты наиблее склонны к возврату кредита в срок.

- Как разные цели кредита влияют на его возврат в срок?

Сгруппириуем значения задолженности по цели взятия клиентом кредита и возьмём средне значение в каждой из категорий

In [44]:
df[['purpose_categ','debt']].groupby('purpose_categ').mean()

,debt
purpose_categ,
автомобиль,0.093590
жилье,0.069058
коммерческая недвижимость,0.074634
образование,0.092200
свадьба,0.080034


# **Вывод**

Более ответственными являются клиенты, берущие кредит для операция с недвижимостью.

# Шаг 4. Общий вывод

Были обнаружены и заменены медианными значениями пропуски в таблице, так же проведена работа с "артефактами". Возможные причины их появления в таблице - нехватка информации от заказчика. Вещественные типы данных были переведены в целочисленные, так же из таблицы были удалены все дубликаты. Мы лемматизировали цели кредита и структурировали в более читабельный вид информацию. Были проанализирлванны зависимости между различными факторами и кредитоспособностью клиентов банка.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x] заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x] есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x] есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.